# **Code for GUI**

### **app.py**

In [ ]:
from flask import Flask, render_template, request
import joblib
import pandas as pd
import logging

# Initialize Flask application
app = Flask(__name__)

# Load pre-trained Random Forest model and encoders
rf_model = joblib.load('random_forest_obesity_model.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Configure logging
logging.basicConfig(level=logging.INFO)

# Define the columns (features) expected by the model
FEATURES = [
    'Gender', 'Age', 'Height', 'Weight', 'Family History with Overweight',
    'Frequent consumption of high caloric food', 'Frequency of consumption of vegetables',
    'Number of main meals', 'Consumption of food between meals', 'Smoke',
    'Consumption of water daily', 'Calories consumption monitoring',
    'Physical activity frequency', 'Time using technology devices',
    'Consumption of alcohol', 'Transportation used'
]

@app.route('/')
def home():
    return render_template('index.html')  # The form will be here

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        try:
            # Log that we received a POST request
            logging.info("Received POST request for prediction")

            # Get the form data
            input_data = [
                request.form.get('gender', ''),
                float(request.form.get('age', 0)),
                float(request.form.get('height', 0)),
                float(request.form.get('weight', 0)),
                request.form.get('family_history', ''),
                request.form.get('caloric_food', ''),
                request.form.get('veggies', ''),
                request.form.get('meals', ''),
                request.form.get('food_between_meals', ''),
                request.form.get('smoke', ''),
                request.form.get('water', ''),
                request.form.get('calories_monitoring', ''),
                request.form.get('activity', ''),
                request.form.get('technology', ''),
                request.form.get('alcohol', ''),
                request.form.get('transport', '')
            ]

            # Log the input data
            logging.info("Input data: %s", input_data)

            # Create a DataFrame for the input
            input_df = pd.DataFrame([input_data], columns=FEATURES)

            # Encode the categorical variables using the saved label encoders
            for column in input_df.columns:
                if column in label_encoders:
                    try:
                        input_df[column] = label_encoders[column].transform([input_df[column][0]])
                    except ValueError:
                        logging.error("Unseen label for column %s: %s", column, input_df[column][0])
                        # You may want to set a default value or handle it accordingly
                        input_df[column] = label_encoders[column].transform(['default_value'])  # Replace 'default_value' with a suitable default

            # Log the encoded input DataFrame
            logging.info("Encoded input data: %s", input_df)

            # Make prediction using the Random Forest model
            rf_prediction = rf_model.predict(input_df)

            # Log the raw prediction result
            logging.info("Raw model prediction: %s", rf_prediction)

            # Decode the prediction
            rf_prediction = label_encoders['Obesity'].inverse_transform(rf_prediction)[0]

            # Log the decoded prediction
            logging.info("Decoded prediction: %s", rf_prediction)

            # Return the result to the result page
            return render_template('result.html', rf_prediction=rf_prediction)

        except Exception as e:
            logging.error("Error during prediction: %s", e)
            # Render home page with an error message if something goes wrong
            return render_template('index.html', error="An error occurred during prediction. Please try again.")

if __name__ == '__main__':
    app.run(debug=True)


### **index.html**

In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Obesity Detection</title>
    <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@400;600&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Montserrat', sans-serif;
            background-image: url('https://images5.alphacoders.com/133/1330530.png');
            background-size: cover;
            background-position: center;
            background-repeat: no-repeat;
            background-color: #e0f7fa;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            margin: 0;
            flex-direction: column;
            color: #3e2723;
        }

        h2 {
            text-align: center;
            color: #003366;
            font-size: 28px;
            font-weight: 600;
            margin-bottom: 30px;
            text-transform: uppercase;
            letter-spacing: 1.5px;
        }

        .container {
            background-color: #ffffff;
            border-radius: 12px;
            padding: 60px;
            box-shadow: 0 10px 25px rgba(0, 0, 0, 0.1);
            width: 700px;
            transition: all 0.3s ease;
        }

        .container:hover {
            box-shadow: 0 15px 30px rgba(0, 0, 0, 0.15);
        }

        .form-row {
            display: flex;
            justify-content: space-between;
            gap: 20px;
            margin-bottom: 20px;
        }

        .height-input {
            width: 100%;
            padding: 10px 15px;
            border: 1px solid #ccc;
            border-radius: 8px;
            font-size: 14px;
            outline: none;
            transition: border-color 0.3s ease;
            margin-bottom: 20px;
        }

        label {
            font-size: 14px;
            color: #666;
            font-weight: 600;
            display: block;
            margin-bottom: 8px;
        }

        .left-column {
            flex: 1;
            text-align: left; /* Align labels to the left */
        }

        .right-column {
            flex: 1;
            text-align: left; /* Align labels to the left */
        }

        input[type="text"],
        select {
            width: 100%;
            padding: 10px 15px;
            border: 1px solid #ccc;
            border-radius: 8px;
            font-size: 14px;
            outline: none;
            transition: border-color 0.3s ease;
            margin-bottom: 10px;
        }

        input[type="text"]:focus,
        select:focus {
            border-color: #007bff;
        }

        select {
            appearance: none;
            background: url('data:image/svg+xml;charset=US-ASCII,%3Csvg xmlns%3D"http%3A//www.w3.org/2000/svg" viewBox%3D"0 0 4 5"%3E%3Cpath fill%3D"%23007bff" d%3D"M2 0L0 2h4L2 0zM0 3h4L2 5 0 3z"/%3E') no-repeat right 10px center/8px 10px;
        }

        .button-container {
            text-align: center;
            margin-top: 20px;
        }

        input[type="submit"],
        button {
            background-color: #007bff;
            color: white;
            border: none;
            padding: 12px 30px;
            font-size: 16px;
            border-radius: 30px;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }


        input[type="submit"]:hover,
        button:hover {
            background-color: #0056b3;
        }

        .home-container {
            text-align: center;
            background-color: #ffffff;
            padding: 40px;
            border-radius: 12px;
            box-shadow: 0 10px 25px rgba(0, 0, 0, 0.1);
            display: none;
        }

        .active {
            display: block;
        }

        .hidden {
            display: none;
        }

        @media (max-width: 768px) {
            .container {
                width: 90%;
                padding: 30px;
            }

            h2 {
                font-size: 24px;
            }

            .form-row {
                flex-direction: column;
            }

            input[type="text"],
            select {
                padding: 8px 12px;
            }
        }
    </style>
</head>

<body>

    <!-- Home Page Section -->
    <div id="homePage" class="home-container active">
        <h2>Obesity Detection System</h2>
        <p>Welcome to the Obesity Detection Application!</p>
        <p>This tool helps to classify your obesity level based on various health metrics.</p>
        <div class="button-container">
            <button onclick="showForm()">Get Started</button>
        </div>
    </div>

    <!-- Form Section -->
    <div id="formPage" class="container hidden">
        <h2>Add Your Info</h2>
        <form id="obesityForm" action="/predict" method="post" onsubmit="return validateForm()">
            <div class="form-row">
                <div class="left-column">
                    <label for="gender">Gender:</label>
                    <select name="gender" id="gender" required>
                        <option value="" disabled selected>Select Gender</option>
                        <option value="Male">Male</option>
                        <option value="Female">Female</option>
                    </select>
                </div>

                <div class="right-column">
                    <label for="age">Age:</label>
                    <input type="text" name="age" id="age" required placeholder="Enter your age">
                </div>
            </div>

            <div class="form-row">
                <div class="left-column">
                    <label for="height">Height (cm):</label>
                    <input type="text" name="height" id="height" required placeholder="Enter your height" class="height-input">
                </div>

                <div class="right-column">
                    <label for="weight">Weight (kg):</label>
                    <input type="text" name="weight" id="weight" required placeholder="Enter your weight">
                </div>
            </div>

            <div class="form-row">
                <div class="left-column">
                    <label for="family_history">Family History with Overweight:</label>
                    <select name="family_history" id="family_history" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="yes">Yes</option>
                        <option value="no">No</option>
                    </select>
                </div>

                <div class="right-column">
                    <label for="caloric_food">Frequent consumption of high caloric food:</label>
                    <select name="caloric_food" id="caloric_food" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="yes">Yes</option>
                        <option value="no">No</option>
                    </select>
                </div>
            </div>

            <div class="form-row">
                <div class="left-column">
                    <label for="veggies">Frequency of consumption of vegetables:</label>
                    <select name="veggies" id="veggies" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="Never">Never</option>
                        <option value="Sometimes">Sometimes</option>
                        <option value="Always">Always</option>
                    </select>
                </div>

                <div class="right-column">
                    <label for="meals">Number of main meals:</label>
                    <select name="meals" id="meals" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="1">1</option>
                        <option value="2">2</option>
                        <option value="3">3</option>
                        <option value="3+">3+</option>
                    </select>
                </div>
            </div>

            <div class="form-row">
                <div class="left-column">
                    <label for="food_between_meals">Consumption of food between meals:</label>
                    <select name="food_between_meals" id="food_between_meals" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="Never">Never</option>
                        <option value="Sometimes">Sometimes</option>
                        <option value="Always">Always</option>
                    </select>
                </div>

                <div class="right-column">
                    <label for="smoke">Smoke:</label>
                    <select name="smoke" id="smoke" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="no">No</option>
                        <option value="yes">Yes</option>
                    </select>
                </div>
            </div>


            <div class="form-row">
                <div class="left-column">
                    <label for="water">Consumption of water daily:</label>
                    <select name="water" id="water" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="Less than a liter">Less than a liter</option>
                        <!--<option value="1-2 L">Between 1 and 2 L</option>-->
                        <option value="More than 2 L">More than 2 L</option>
                    </select>
                </div>

                <div class="right-column">
                    <label for="calories_monitoring">Calories consumption monitoring:</label>
                    <select name="calories_monitoring" id="calories_monitoring" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="yes">Yes</option>
                        <option value="no">No</option>
                        </select>
                </div>
            </div>

            <div class="form-row">
                <div class="left-column">
                    <label for="activity">Physical activity frequency:</label>
                    <select name="activity" id="activity" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="I do not have">I do not have</option>
                        <option value="1 or 2 days">1 or 2 days</option>
                        <option value="2 or 4 days">2 or 4 days</option>
                        <option value="4 or 5 days">4 or 5 days</option>
                    </select>
                </div>

                <div class="right-column">
                    <label for="technology">Time using technology devices:</label>
                    <select name="technology" id="technology" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="0–2 hours">0–2 hours</option>
                        <option value="3–5 hours">3–5 hours</option>
                        <option value="More than 5 hours">More than 5 hours</option>
                    </select>
                </div>
            </div>

            <div class="form-row">
                <div class="left-column">
                    <label for="alcohol">Consumption of alcohol:</label>
                    <select name="alcohol" id="alcohol" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="no">No</option>
                        <option value="Sometimes">Sometimes</option>
                        <option value="Frequently">Frequently</option>
                    </select>
                </div>

                <div class="right-column">
                    <label for="transport">Transportation used:</label>
                    <select name="transport" id="transport" required>
                        <option value="" disabled selected>Select Option</option>
                        <option value="Public Transportation">Public Transportation</option>
                        <option value="Automobile">Automobile</option>
                        <option value="Motorbike">Motorbike</option>
                        <option value="Bike">Bike</option>
                        <option value="Walking">Walking</option>
                    </select>
                </div>
            </div>



            <div class="button-container">
                <input type="submit" value="Submit">
                <button type="button" onclick="showHome()">Return to Home Page</button>
            </div>
        </form>
    </div>

    <script>
        function showForm() {
            document.getElementById('homePage').classList.add('hidden');
            document.getElementById('formPage').classList.remove('hidden');
        }

        function showHome() {
            document.getElementById('formPage').classList.add('hidden');
            document.getElementById('homePage').classList.remove('hidden');
        }

        function validateForm() {
    let age = document.getElementById("age").value;
    let height = document.getElementById("height").value;
    let weight = document.getElementById("weight").value;

    // Age validation: Must be a number between 1 and 120
    if (!isNumeric(age) || age < 1 || age > 120) {
        alert("Please enter a valid age between 1 and 120.");
        return false;
    }

    // Height validation: Must be a number between 50 cm and 250 cm
    if (!isNumeric(height) || height < 50 || height > 250) {
        alert("Please enter a valid height between 50 and 250 cm.");
        return false;
    }

    // Weight validation: Must be a number between 10 kg and 300 kg
    if (!isNumeric(weight) || weight < 10 || weight > 300) {
        alert("Please enter a valid weight between 10 and 300 kg.");
        return false;
    }

    return true;
}

function isNumeric(value) {
    return /^\d+(\.\d+)?$/.test(value);  // Allows integers and decimals
}


    </script>

</body>

</html>

### **result.html**

In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Obesity Prediction</title>
    <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@400;600&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Montserrat', sans-serif;
            background-color: #e0f7fa; /* Light cyan background */
            background-image: url('https://inlarkin.com/cdn/shop/articles/10_Tiny_Habits_to_Improve_Your_Health_Weath_and_Happiness_Inlarkin_Blog_1024x.png?v=1613521751s'); /* Add the path to your image here */
            background-size: cover; /* Cover the entire viewport */
            background-position: center; /* Center the image */
            color: #3e2723; /* Dark brown text color */
            margin: 0;
            padding: 20px;
            display: flex;
            flex-direction: column;
            align-items: center;
            height: 100vh;
        }

        h2 {
            text-align: center;
            color: #003366; /* Navy blue for headings */
            margin-bottom: 30px;
            font-size: 28px;
            font-weight: 600;
            text-transform: uppercase;
            letter-spacing: 1.5px;
        }

        .result-container {
            background-color: #ffffff; /* White background for result container */
            padding: 30px;
            border-radius: 12px;
            box-shadow: 0 10px 25px rgba(0, 0, 0, 0.1);
            max-width: 600px;
            text-align: center;
        }

        .result {
            margin: 15px 0;
            font-size: 20px;
            font-weight: bold;
        }

        .prediction {
            font-size: 24px;
            font-weight: bold;
            text-transform: uppercase;
        }

        .normal-weight {
            color: green; /* Green color for Normal Weight */
        }

        .other-prediction {
            color: red; /* Red color for other categories */
        }

        a {
            display: inline-block;
            margin-top: 20px;
            text-decoration: none;
            color: #007bff; /* Bright blue for links */
            font-weight: bold;
            padding: 10px 20px;
            border-radius: 30px;
            background-color: #e3f2fd;
            transition: background-color 0.3s, color 0.3s;
        }

        a:hover {
            background-color: #bbdefb;
            color: #004d40;
        }

        /* Responsive adjustments */
        @media (max-width: 768px) {
            .result-container {
                padding: 20px;
            }

            h2 {
                font-size: 24px;
            }

            .result {
                font-size: 18px;
            }

            .prediction {
                font-size: 20px;
            }
        }
    </style>
</head>

<body>
    <h2>Obesity classification Results</h2>
    <div class="result-container">
        <div class="result">Based on the classification result, you belong to the following category:</div>
        <div class="prediction {% if rf_prediction == 'Normal Weight' %}normal-weight{% else %}other-prediction{% endif %}">
            {{ rf_prediction | upper }}
        </div>
        <a href="/">Back to Home</a>
    </div>
</body>

</html>
